In [1]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import mido
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np
from pretty_midi import PrettyMIDI
import midi_utils as mu
#import scales

In [12]:
MIDI_FOLDER = "MIDI_Files/"

# filename = "KCP_Major_1.mid"
# filename = "Under-The-Sea.mid"
# filename = "Wii Channels - Mii Channel.mid"
# filename = "mii_channel_reexported.mid"
# filename = "test_piano.mid"
# filename = "chpn-p1.mid"
# filename = "HesaPirate.mid"
# filename = "toto-africa.mid"
# filename = "vivconct.mid"
# filename = "take_on_me_aha.mid"
filename = "Never-Gonna-Give-You-Up-3.mid"
m = mido.MidiFile(filename=MIDI_FOLDER + filename)
minf = mu.get_midi_infos(m)
minf = mu.filter_midi_infos(minf)
minf = mu.dispatch_midi_infos_by_channel(minf, track_name_method="accumulate")
for k,v in minf.items():
    if k != "tracks_info":
        print(k, ":", v)
    else:
        for x in v:
            print(x)
            
# for x in m:
#     print(x)

info_type : filtered_dispatched
filename : MIDI_Files/Never-Gonna-Give-You-Up-3.mid
midi_type : 0
track_count : 16
music_track_count : 16
ticks_per_beat : 384
length : 216.2045180000046
{'name': '0013GIVE Channel 1', 'meta_only': False, 'channel_count': {1: 37}, 'cc_count': {1: 13}, 'meta_count': 0, 'typeset': {'control_change', 'pitchwheel', 'note_on', 'note_off', 'program_change'}, 'get_track': <function get_midi_track_info.<locals>.<lambda> at 0x000001DBAB7C8160>}
{'name': '0013GIVE Channel 2', 'meta_only': False, 'channel_count': {2: 1662}, 'cc_count': {2: 13}, 'meta_count': 0, 'typeset': {'note_on', 'control_change', 'note_off', 'program_change'}, 'get_track': <function get_midi_track_info.<locals>.<lambda> at 0x000001DBAB7B3DC0>}
{'name': '0013GIVE Channel 3', 'meta_only': False, 'channel_count': {3: 1008}, 'cc_count': {3: 13}, 'meta_count': 0, 'typeset': {'note_on', 'control_change', 'note_off', 'program_change'}, 'get_track': <function get_midi_track_info.<locals>.<lambda> at 0

In [ ]:
# music = PrettyMIDI(midi_file=filename)
# bla = music.synthesize(fs=22050)
# print(bla[40000:50000])
# sd.play(bla, 22050)

In [ ]:
def track_to_dataframe(track, event_style=False):
    records = []
    track_name = ""
    current_time = 0
    # Event style dataframe
    if event_style:
        for x in track:
            new_dict = x.__dict__.copy()
            if not x.is_meta():
                #Swapping name for coherence
                new_dict["time_diff"] = new_dict["time"] 
                current_time += new_dict["time_diff"]
                new_dict["time"] = current_time
                new_dict["pressed"] = new_dict["type"] == "note_on"
                new_dict["released"] = new_dict["type"] == "note_off"
                records.append(new_dict)
    else:
        # Streaming style dataframe
        pressed_notes = {}
        id = 0
        for x in track:
            new_dict = x.__dict__.copy()
            if new_dict["type"] == "track_name":
                track_name = new_dict["name"]
            else:
                current_time += new_dict["time"]
                new_dict["time"] = current_time
                new_dict["time_release"] = None
                new_dict["time_duration"] = None
                new_dict["velocity_release"] = None
                if new_dict["type"] == "note_off":
                    former_pressed_note = pressed_notes[new_dict["note"]]
                    if former_pressed_note is None:
                        raise ValueError("The given track has a released note that was never pressed in the first place")
                    pressed_record = records[former_pressed_note["id"]]
                    pressed_record["time_release"] = current_time
                    pressed_record["time_duration"] = current_time - pressed_record["time"]
                    pressed_record["velocity_release"] = new_dict["velocity"]
                    pressed_notes[new_dict["note"]] = None
                elif new_dict["type"] == "note_on":
                    pressed_notes[new_dict["note"]] = {"id": id}
                    del new_dict["type"]
                    records.append(new_dict)
                    id += 1
                                        
    return pd.DataFrame(records)

pd_track = track_to_dataframe(m.tracks[1])
print(pd_track)

In [ ]:
def plot_music(scheduled_style_df: pd.DataFrame, chroma_plot=False, ax=None, cmap=plt.get_cmap("gist_rainbow")):
    if ax is None:
        _, ax = plt.subplots()
    
    df_copy = scheduled_style_df[["time", "note", "time_duration", "time_release"]]
    if chroma_plot:
        df_copy.loc[:, "note"] = mu.to_chroma(df_copy["note"])
    for i, x in df_copy.iterrows():
        rect = patches.Rectangle((x.time, x.note - 0.5), width=x.time_duration, height=1, linewidth=0.2, edgecolor=(0,0,0), facecolor=cmap(x.note))
        ax.add_patch(rect)
    plt.xlim(0, df_copy.time_release.max())
    plt.ylim(df_copy.note.min() - 3, df_copy.note.max() + 3)

plot_music(pd_track, chroma_plot=False)

In [ ]:
midi_ids = pd_track.note.to_numpy()

# Transposition:
transposition = 0
midi_ids += (transposition%12)

# general_scale_subset = scales.create_general_scale_subset()
general_scale_subset = scales.create_general_scale_subset(note_counts=[5,7])

print(len(general_scale_subset), "general scales will be examined")
music_note_count = len(midi_ids)
print("Music note count:", music_note_count)
window_size = max(int(music_note_count * 0.1), 14)
print("Window size:", window_size)

print(scales.windowed_suggest_scales(midi_ids, threshold=0.9, window_size=window_size, window_threshold=0.9, general_scale_subset=general_scale_subset))